In [ ]:
# This Jupyter notebook is for DRN univariate post-processing on temperature forecasts,
# using the data from https://doi.org/10.6084/m9.figshare.19453580
# The script is an adapted version of the replication Jupyter notebook from https://github.com/slerch/ppnn

In [1]:
import pandas as pd
import pyarrow
import numpy as np

In [16]:
# read data

data = pd.read_feather(path = "./temperature_data_cgm_std.feather")
data = data.iloc[:, list(range(0,3))+list(range(53,91))]
data.station = pd.to_numeric(data.station, downcast = 'integer')

# split into train and test data
eval_start = 1626724
train_end = 1626724

train_features_raw = data.iloc[:train_end,3:].to_numpy()
train_targets = data.iloc[:train_end,2].to_numpy()
train_IDs = data.iloc[:train_end,1].to_numpy()

test_features_raw = data.iloc[eval_start:,3:].to_numpy()
test_targets = data.iloc[eval_start:,2].to_numpy()
test_IDs = data.iloc[eval_start:,1].to_numpy()

In [18]:
# normalize data

def normalize(data, method=None, shift=None, scale=None):
    result = np.zeros(data.shape)
    if method == "MAX":
        scale = np.max(data, axis=0)
        shift = np.zeros(scale.shape)
    for index in range(len(data[0])):
        result[:,index] = (data[:,index] - shift[index]) / scale[index]
    return result, shift, scale

train_features, train_shift, train_scale = normalize(train_features_raw[:,:], method="MAX")

test_features = normalize(test_features_raw[:,:], shift=train_shift, scale=train_scale)[0]

In [19]:
# helper functions for NN models

import keras
import keras.backend as K
import tensorflow as tf

from keras.layers import Input, Dense, merge, Embedding, Flatten, Concatenate
from keras.models import Model, Sequential
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping


def crps_cost_function(y_true, y_pred, theano=False):
    """Compute the CRPS cost function for a normal distribution defined by
    the mean and standard deviation.

    Code inspired by Kai Polsterer (HITS).

    Args:
        y_true: True values
        y_pred: Tensor containing predictions: [mean, std]
        theano: Set to true if using this with pure theano.

    Returns:
        mean_crps: Scalar with mean CRPS over batch
    """

    # Split input
    mu = y_pred[:, 0]
    sigma = y_pred[:, 1]
    # Ugly workaround for different tensor allocation in keras and theano
    if not theano:
        y_true = y_true[:, 0]   # Need to also get rid of axis 1 to match!

    # To stop sigma from becoming negative we first have to 
    # convert it the the variance and then take the square
    # root again. 
    var = K.square(sigma)
    # The following three variables are just for convenience
    loc = (y_true - mu) / K.sqrt(var)
    phi = 1.0 / np.sqrt(2.0 * np.pi) * K.exp(-K.square(loc) / 2.0)
    Phi = 0.5 * (1.0 + tf.math.erf(loc / np.sqrt(2.0)))
    # First we will compute the crps for each input/target pair
    crps =  K.sqrt(var) * (loc * (2. * Phi - 1.) + 2 * phi - 1. / np.sqrt(np.pi))
    # Then we take the mean. The cost is now a scalar
    return K.mean(crps)

def build_emb_model(n_features, n_outputs, hidden_nodes, emb_size, max_id,
                    compile=False, optimizer='adam', lr=0.01,
                    loss=crps_cost_function,
                    activation='relu', reg=None):
    """

    Args:
        n_features: Number of features
        n_outputs: Number of outputs
        hidden_nodes: int or list of hidden nodes
        emb_size: Embedding size
        max_id: Max embedding ID
        compile: If true, compile model
        optimizer: Name of optimizer
        lr: learning rate
        loss: loss function
        activation: Activation function for hidden layer

    Returns:
        model: Keras model
    """
    if type(hidden_nodes) is not list:
        hidden_nodes = [hidden_nodes]

    features_in = Input(shape=(n_features,))
    id_in = Input(shape=(1,))
    emb = Embedding(max_id + 1, emb_size)(id_in)
    emb = Flatten()(emb)
    x = Concatenate()([features_in, emb])
    for h in hidden_nodes:
        x = Dense(h, activation=activation, kernel_regularizer=reg)(x)
    x = Dense(n_outputs, activation='linear', kernel_regularizer=reg)(x)
    model = Model(inputs=[features_in, id_in], outputs=x)

    if compile:
        opt = keras.optimizers.Adam(lr=lr)
        model.compile(optimizer=opt, loss=loss)
    return model

In [107]:
# training multiple models in a loop

emb_size = 2
max_id = int(np.max([train_IDs.max(), test_IDs.max()]))
n_features = train_features.shape[1]
n_outputs = 2

nreps = 10
trn_scores = []
test_scores = []
preds = []
repred = []

from tqdm.notebook import tqdm

for i in tqdm(range(nreps)):
    
    tf.compat.v1.reset_default_graph()
    keras.backend.clear_session()
    
    features_in = Input(shape=(n_features,))
    id_in = Input(shape=(1,))
    emb = Embedding(max_id + 1, emb_size)(id_in)
    emb = Flatten()(emb)
    x = Concatenate()([features_in, emb])
    x = Dense(512, activation='relu')(x)
    x = Dense(n_outputs, activation='linear')(x)
    nn_aux_emb = Model(inputs=[features_in, id_in], outputs=x)

    opt = keras.optimizers.Adam(lr=0.002)
    nn_aux_emb.compile(optimizer=opt, loss=crps_cost_function)
    
    nn_aux_emb.fit([train_features, train_IDs], train_targets, epochs=15, batch_size=4096, verbose=0)   
    
    trn_scores.append(nn_aux_emb.evaluate([train_features, train_IDs], train_targets, 4096, verbose=0))
    test_scores.append(nn_aux_emb.evaluate([test_features, test_IDs], test_targets, 4096, verbose=0))
    
    preds.append(nn_aux_emb.predict([test_features, test_IDs], 4096, verbose=0))
    repred.append(nn_aux_emb.predict([train_features, train_IDs], 4096, verbose=0))

  0%|          | 0/10 [00:00<?, ?it/s]

C:\Users\gm2154\Anaconda3\envs\python-mlpp\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
C:\Users\gm2154\Anaconda3\envs\python-mlpp\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
C:\Users\gm2154\Anaconda3\envs\python-mlpp\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
C:\Users\gm2154\Anaconda3\envs\python-mlpp\lib\site-packag

In [108]:
preds = np.array(preds)
preds[:, :, 1] = np.abs(preds[:, :, 1]) # Make sure std is positive
mean_preds = np.mean(preds, 0)

repred = np.array(repred)
repred[:, :, 1] = np.abs(repred[:, :, 1]) # Make sure std is positive
mean_repred = np.mean(repred, 0)

array([[ 4.360059  ,  0.6530824 ],
       [ 4.589731  ,  0.7056192 ],
       [ 4.066346  ,  0.61987317],
       ...,
       [ 6.9508705 ,  1.4646157 ],
       [ 1.3789208 ,  1.3837237 ],
       [-1.6033599 ,  1.7533748 ]], dtype=float32)

In [109]:
train_info = data.iloc[:train_end,:2]
test_info = data.iloc[eval_start:,:2]

train_info = train_info.reset_index()
test_info = test_info.reset_index()

mean_preds_df = pd.DataFrame(mean_preds)
mean_repred_df = pd.DataFrame(mean_repred)

In [110]:
train_combine = pd.concat([train_info, mean_repred_df], axis=1) 
test_combine = pd.concat([test_info, mean_preds_df], axis=1) 

In [112]:
train_combine.to_csv('./nn_t2m_train.csv', index=False)
test_combine.to_csv('./nn_t2m_test.csv', index=False)